In [1]:
# Bibliotecas principais
import numpy  as np
import pandas as pd


#  Bibliotecas para Corpus processing
import re
import nltk.corpus
from unidecode                        import unidecode
from nltk.tokenize                    import word_tokenize
from nltk                             import SnowballStemmer
from sklearn.feature_extraction.text  import TfidfVectorizer
from sklearn.feature_extraction.text  import CountVectorizer
from sklearn.preprocessing            import normalize

In [2]:
#nltk.download('punkt')
#nltk.download('stopwords')

In [3]:
# Importando arquivo csv e visualizando as 6 primeiras linhas
data = pd.read_csv('bbc_news_mixed.csv', sep = ',', encoding = 'latin 1')
data.head(6)

,text,label
0,Cairn shares slump on oil setback\n\nShares in...,business
1,Egypt to sell off state-owned bank\n\nThe Egyp...,business
2,Cairn shares up on new oil find\n\nShares in C...,business
3,Low-cost airlines hit Eurotunnel\n\nChannel Tu...,business
4,"Parmalat to return to stockmarket\n\nParmalat,...",business
5,Making your office work for you\n\nOur mission...,business


In [4]:
# Transformar a coluna desejada do data frame em uma lista e visualizando o 9º elemento
corpus = data['text'].to_list() 
corpus[8]

'German bidder in talks with LSE\n\nDeutsche Boerse bosses have held "constructive, professional and friendly" talks with the London Stock Exchange (LSE), its chief has said.\n\nWerner Seifert met LSE chief executive Clara Furse amid rumours the German group may raise its bid to Â£1.5bn ($2.9bn) from its initial Â£1.3bn offer. However, rival suitor Euronext also upped the ante in the bid battle. Ahead of talks with the LSE on Friday, the pan-European bourse said it may be prepared to make its offer in cash. The Paris-based exchange, owner of Liffe in London, is reported to be ready to raise Â£1.4bn to fund a bid.\n\nThe news came as Deutsche Boerse held its third meeting with the LSE since its bid approach in December which was turned down by the London exchange for undervaluing the business. However, the LSE did agree to leave the door open for talks to find out whether a "significantly-improved proposal" would be in the interests of LSE\'s shareholders and customers. In the meantime,

In [5]:
def removeWords(listOfTokens, listOfWords):
    return [token for token in listOfTokens if token not in listOfWords]

# applies stemming to a list of tokenized words
def applyStemming(listOfTokens, stemmer):
    return [stemmer.stem(token) for token in listOfTokens]

# removes any words composed of less than 2 or more than 21 letters
def twoLetters(listOfTokens):
    twoLetterWord = []
    for token in listOfTokens:
        if len(token) <= 2 or len(token) >= 21:
            twoLetterWord.append(token)
    return twoLetterWord

In [6]:
def processCorpus(corpus, language):   
    stopwords = nltk.corpus.stopwords.words(language)
    param_stemmer = SnowballStemmer(language)
    
    for document in corpus:
        index = corpus.index(document)
        #corpus[index] = corpus[index].replace(u'\ufffd', '8')   # Replaces the ASCII '�' symbol with '8'
        corpus[index] = corpus[index].replace(',', '')          # Removes commas
        corpus[index] = corpus[index].rstrip('\n')              # Removes line breaks
        corpus[index] = corpus[index].casefold()                # Makes all letters lowercase
        
        corpus[index] = re.sub('\W_',' ', corpus[index])        # removes specials characters and leaves only words
        corpus[index] = re.sub("\S*\d\S*"," ", corpus[index])   # removes numbers and words concatenated with numbers IE h4ck3r. Removes road names such as BR-381.
        corpus[index] = re.sub("\S*@\S*\s?"," ", corpus[index]) # removes emails and mentions (words with @)
        corpus[index] = re.sub(r'http\S+', '', corpus[index])   # removes URLs with http
        corpus[index] = re.sub(r'www\S+', '', corpus[index])    # removes URLs with www

        listOfTokens = word_tokenize(corpus[index])
        twoLetterWord = twoLetters(listOfTokens)
        
        listOfTokens = removeWords(listOfTokens, stopwords)
        listOfTokens = removeWords(listOfTokens, twoLetterWord)
        
        listOfTokens = applyStemming(listOfTokens, param_stemmer)

        corpus[index]   = " ".join(listOfTokens)
        corpus[index] = unidecode(corpus[index])

    return corpus

In [7]:
language = 'english'
corpus = processCorpus(corpus, language)
corpus[8]

'german bidder talk lse deutsch boers boss held construct profession friend talk london stock exchang lse chief said werner seifert met lse chief execut clara furs amid rumour german group may rais bid initi offer howev rival suitor euronext also up ant bid battl ahead talk lse friday pan-european bours said may prepar make offer cash paris-bas exchang owner liff london report readi rais fund bid news came deutsch boers held third meet lse sinc bid approach decemb turn london exchang undervalu busi howev lse agre leav door open talk find whether propos would interest lse sharehold custom meantim euronext combin pari amsterdam lisbon stock exchang also began talk lse statement thursday euronext said offer like sole cash ad assur stage offer made deal either bidder would creat biggest stock market oper europ second biggest world new york stock exchang howev neither side made formal offer lse sourc claim step may still week away deutsch boers could also face mount opposit bid home among s

In [8]:
# Aplicando o TFIDF
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print("X.shape: ",X.shape)
count_vec = pd.DataFrame(data = X.toarray(), columns=vectorizer.get_feature_names())

final_df = count_vec

print("{} linhas".format(final_df.shape[0])) 

# As 5 primeiras palavras coom maior peso no documento 9
final_df.T.nlargest(5, 8)

X.shape:  (2225, 19157)
2225 linhas


,0,1,2,3,4,5,6,7,8,9,...,2215,2216,2217,2218,2219,2220,2221,2222,2223,2224
lse,0,0,0,0,0,0,0,0,9,0,...,0,0,0,0,0,0,0,0,0,0
also,3,0,2,0,0,1,0,0,6,1,...,2,0,3,1,1,0,3,0,2,2
bid,0,0,0,0,0,0,0,0,5,0,...,0,0,0,0,0,0,0,1,0,0
boers,0,0,0,0,0,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,0
exchang,0,1,0,0,2,0,0,2,5,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# visualizando a matriz tfidf gerada
final_df

,aaa,aac,aadc,aaliyah,aaltra,aamir,aan,aara,aarhus,aaron,...,zoom,zooropa,zornotza,zorro,zubair,zuluaga,zurich,zuton,zvonareva,zvyagintsev
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2220,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2221,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2222,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2223,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
data.label

0       business
1       business
2       business
3       business
4       business
          ...   
2220        tech
2221        tech
2222        tech
2223        tech
2224        tech
Name: label, Length: 2225, dtype: object

In [11]:
data.label.value_counts()

sport            511
business         510
politics         417
tech             401
entertainment    386
Name: label, dtype: int64

In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data["resposta"] = pd.Series(le.fit_transform(data.label))

In [13]:
result = pd.concat([final_df, data["resposta"]], axis=1)

In [14]:
result.resposta.value_counts()

3    511
0    510
2    417
4    401
1    386
Name: resposta, dtype: int64

In [15]:
result

,aaa,aac,aadc,aaliyah,aaltra,aamir,aan,aara,aarhus,aaron,...,zooropa,zornotza,zorro,zubair,zuluaga,zurich,zuton,zvonareva,zvyagintsev,resposta
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2220,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2221,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2222,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2223,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


In [16]:
X = result.drop('resposta', axis=1)
y = result["resposta"]

In [18]:
!install xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state=42)
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

'install' não é reconhecido como um comando interno
ou externo, um programa operável ou um arquivo em lotes.


ModuleNotFoundError: No module named 'xgboost'

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0),"\n")
conf_mat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print('Confusion matrix:\n', conf_mat)
print('\nClassification Report:\n',classification_report(y_test, y_pred))

In [ ]:
from matplotlib import pyplot as plt
labels = ['Class 0', 'Class 1','Class 2','Class 3', 'Class 4']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(conf_mat, cmap=plt.cm.Blues)
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Classe Predita')
plt.ylabel('Classe Real')
plt.show()

In [ ]:
y_pred = model.predict(X)
accuracy = accuracy_score(y, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0),"\n")
conf_mat = confusion_matrix(y_true=y, y_pred=y_pred)
print('Confusion matrix:\n', conf_mat)
print('\nClassification Report:\n',classification_report(y, y_pred))

In [ ]:
model_completo = XGBClassifier()
model_completo.fit(X,y)
model_completo.predict(X)
